In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

def SCD2function():
    catalog_name = "streaming1"
    db_name = "silver"
    table_name_stage = "stg_product"
    table_name_dim = "dim_product"

    column_name1="id"
    column_name2 = "body_html"
    column_name3 = "title"
    column_name4 = "handle"
    column_name5 = "product_type"
    column_name6 = "vendor"
    column_name7 = "created_at"
    column_name8 = "status"


    db_name_source = "_fivetran_setup_test"
    table_name_source = "product"
    catalog_source = "hive_metastore"


    df=spark.sql(f"use catalog {catalog_name}")
    df=spark.sql(f"use schema {db_name};")

    spark.sql(f"drop table if exists {table_name_stage};")

    spark.sql(f"create table {table_name_stage} ({column_name1} string, {column_name2} string, {column_name3} string, {column_name4} string, {column_name5} string, {column_name6} string, {column_name7} string, {column_name8} string) using delta;")

    spark.sql(f"truncate table {table_name_stage};")
    spark.sql(f"insert into {table_name_stage} select TRY_CAST({column_name1} as string) as id_, TRY_CAST({column_name2} as string),TRY_CAST({column_name3} as string),TRY_CAST({column_name4} as string),TRY_CAST({column_name5} as string),TRY_CAST({column_name6} as string),TRY_CAST({column_name7} as string),TRY_CAST({column_name8} as string) from {catalog_source}.{db_name_source}.{table_name_source};")

    df=spark.sql(f"merge into {table_name_dim} a using (with q1 as (select * from {table_name_dim} where {column_name8}<>'inactive') select {column_name1} as mergeKey,* from {table_name_stage} union all select NULL as mergeKey, a.* from {table_name_stage} a join q1 b on a.{column_name1}=b.{column_name1} and b.{column_name1}<>'inactive' and (a.{column_name2}!=b.{column_name2} or a.{column_name3}!=b.{column_name3} or a.{column_name4}!=b.{column_name4} or a.{column_name5}!=b.{column_name5} or a.{column_name6}!=b.{column_name6} or a.{column_name7}!=b.{column_name7} ) ) b on a.{column_name1}=b.mergeKey WHEN MATCHED and (b.{column_name2} !=a.{column_name2} or b.{column_name3} !=a.{column_name3} or b.{column_name4} !=a.{column_name4} or b.{column_name5} !=a.{column_name5} or b.{column_name6} !=a.{column_name6} or b.{column_name7} !=a.{column_name7} or b.{column_name8} !=a.{column_name8}) then update set {column_name8}='inactive' ,a.end_date=current_date()-1 when NOT MATCHED THEN insert ({column_name1},{column_name2},{column_name3},{column_name4},{column_name5},{column_name6},{column_name7},{column_name8},start_date,end_date) values ({column_name1},{column_name2},{column_name3},{column_name4},{column_name5},{column_name6},{column_name7},{column_name8},current_date(),'9999-12-31');")
    df.display()

In [0]:
SCD2function()

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2,2,0,0


In [0]:
df=spark.sql(f"select * from {table_name_dim};")
display(df)

id,body_html,title,handle,product_type,vendor,created_at,status,start_date,end_date
9433797296426,"The Stan Smith owned the tennis court in the '70s. Today it runs the streets with the same clean, classic style. These kids' shoes preserve the iconic look of the original, made in leather with punched 3-Stripes, heel and tongue logos and lightweight step-in cushioning.",ADIDAS | KID'S STAN SMITH,adidas-kids-stan-smith,SHOES,ADIDAS,2024-08-16 08:37:07,active,2024-09-09,9999-12-31
9437953065258,null,Green Snowboard,green-snowboard,,tylerstorecoding,2024-08-18 00:46:43,active,2024-09-09,9999-12-31
9433802670378,Flexfit Mini Ottoman Cap,FLEX FIT | MINI OTTOMAN BLACK,flex-fit-mini-ottoman-black,ACCESSORIES,FLEX FIT,2024-08-16 08:37:20,active,2024-09-09,9999-12-31
9433781731626,"Make a statement with the iconic Dr Marten's 1461Z DMC 3-Eye shoe. Stamped with Docs trademade look, this lace-up shoe is crafted with high-quality, durable smooth leather, and has been made to last. This unique shoe features an air-cushioned sole offering good abrasion and slip resistance, and is made with Goodyear welt - the sole and upper are heat-sealed and sewn together.",DR MARTENS | 1461 DMC 3-EYE SHOE | BLACK SMOOTH,dr-martens-1461-dmc-3-eye-shoe-black-smooth,SHOES,DR MARTENS,2024-08-16 08:36:27,active,2024-09-09,9999-12-31
9433805652266,"This women's backpack has a glam look, thanks to a faux-leather build with an allover fur print. The front zip pocket keeps small things within reach, while an interior divider reins in potential chaos.",ADIDAS | CLASSIC BACKPACK,adidas-classic-backpack,ACCESSORIES,ADIDAS,2024-08-16 08:37:28,active,2024-09-09,9999-12-31
9433772425514,"Vans are a staple in skate culture and street style, and this skate sneaker is no exception. Introduced in the early 70's, the Vans SK8 Hi was responsible for the reduced number of ankle injuries in the skating scene with the extra padded ankle support, but its popularity has grown across different cultural scenes. The Vans SK8 Hi True White is constructed from a combination of canvas and suede and features the signature side stripe detail and vulcanized waffle sole. Available in a range of colours.",VANS | SH-8 HI,vans-sh-8-hi,SHOES,VANS,2024-08-16 08:36:02,active,2024-09-09,9999-12-31
9437983703338,null,Green Snowboard,green-snowboard-2,,tylerstorecoding,2024-08-18 01:37:21,active,2024-09-09,9999-12-31
9433756303658,"Vans are a staple in skate culture and street style, and the Vans Era 59 is no exception. This classic lace-up skate shoe is focused on ultimate comfort and cool. Constructed from canvas with leather accents which sets it apart from the original Era, it features a soft footbed, double stitched vamp, a padded tongue and lining, as well as Vans' signature waffle sole.",VANS | ERA 59 MOROCCAN | GEO/DRESS BLUES,vans-era-59-moroccan-geo-dress-blues,SHOES,VANS,2024-08-16 08:35:17,active,2024-09-09,9999-12-31
9433778815274,"Timberland has been refining their craft of producing hard-wearing boots for decades. The 6-Inch Premium Waterproof Boot (or Wheat Boot) is the original men's Timberland boot and was first designed nearly 40 years ago. Continuous improvements to the design and construction, such as the addition of Timberland's exclusive anti-fatigue technology for all day comfort, has cemented these boots as a sturdy and dependable staple in anyone's shoe collection. The premium waterproof leather, seam-sealed construction and rugged lug outsoles provides long-lasting durability and comfort to help you get through any environmental conditions and adventure. An absolute must-have.",TIMBERLAND | MENS 6 INCH PREMIUM BOOT,timberland-mens-6-inch-premium-boot,SHOES,TIMBERLAND,2024-08-16 08:36:19,active,2024-09-09,9999-12-31
9433800278314,"Featuring a cinch closure concealed by a slender top flap, the Iona backpack unites everyday practicality with clean design.",HERSCHEL | IONA,herschel-iona,ACCESSORIES,HERSCHEL,2024-08-16 08:37:14,active,2024-09-09,9999-12-31
